<a href="https://colab.research.google.com/github/sumithdcosta/Python/blob/master/RestApi/01-Chapter/RestApi-Flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install flask
!pip install flask-cors
!pip install flask-ngrok
!pip install Flask-RESTful
!pip install Flask-JWT
!pip install 

     |████████████████████████████████| 61kB 7.8MB/s 
  Created wheel for Flask-JWT: filename=Flask_JWT-0.3.2-cp37-none-any.whl size=5670 sha256=eb4409eda8f644c478a103a872f65ba4d877e030012261aae6f4bbe1e78a447b
  Stored in directory: /root/.cache/pip/wheels/a6/8b/1f/f42f18eea64a0884185293ff01c32124c49da0e62739815f51
Successfully built Flask-JWT
ERROR: You must give at least one requirement to install (see "pip help install")


In [ ]:
!pip freeze

In [3]:
!mkdir bin

In [ ]:
%%writefile firstRestAPI.py
from flask_ngrok import run_with_ngrok
from flask import Flask

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')   #root directory
def home():
  return "Hello World !"


if __name__ == '__main__':
  app.run()

Writing firstRestAPI.py


In [ ]:
!python firstRestAPI.py

 * Serving Flask app "firstRestAPI" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Running on http://359beffe5dda.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
^C


In [5]:
%%writefile store.py
from flask_ngrok import run_with_ngrok
from flask import Flask,jsonify,request,render_template

app = Flask(__name__)
run_with_ngrok(app)


stores = [
  {
   'name':'My Store',
   'items':[
      {
        'name':'chair',
        'price':15.99
      }
            
   ]
   }          
]




#post /store data: {name :}
@app.route('/store' , methods=['POST'])
def create_store():
  request_data = request.get_json()
  new_store = {
    'name':request_data['name'],
    'items':[]
  }
  stores.append(new_store)
  return jsonify(new_store)
  #pass

#get /store/<name> data: {name :}
@app.route('/store/<string:name>')
def get_store(name):
  for store in stores:
    if store['name'] == name:
          return jsonify(store)
  return jsonify ({'message': 'store not found'})
  #pass

#get /store
@app.route('/store')
def get_stores():
  return jsonify({'stores': stores})
  #pass

#post /store/<name> data: {name :}
@app.route('/store/<string:name>/item' , methods=['POST'])
def create_item_in_store(name):
  request_data = request.get_json()
  for store in stores:
    if store['name'] == name:
        new_item = {
            'name': request_data['name'],
            'price': request_data['price']
        }
        store['items'].append(new_item)
        return jsonify(new_item)
  return jsonify ({'message' :'store not found'})
  #pass

#get /store/<name>/item data: {name :}
@app.route('/store/<string:name>/item')
def get_item_in_store(name):
  for store in stores:
    if store['name'] == name:
        return jsonify( {'items':store['items'] } )
  return jsonify ({'message':'store not found'})

  #pass
 
if __name__ == '__main__':
  app.run()

Overwriting store.py


In [6]:
!python store.py

 * Serving Flask app "store" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Running on http://8243122fb37c.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
^C


In [16]:
mkdir bin

In [21]:
%%writefile bin/app.py
from flask import Flask
from flask_restful import Resource, Api
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
api = Api(app)
run_with_ngrok(app)

class Student(Resource):
  def get(self, name):
    return {'student': name}

api.add_resource(Student, '/student/<string:name>')

if __name__ == '__main__':
  app.run()

Overwriting bin/app.py


In [22]:
!python bin/app.py

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Running on http://3dee12747a19.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
127.0.0.1 - - [16/May/2021 11:30:37] "GET /student/sumtih HTTP/1.1" 200 -
^C


In [29]:
%%writefile bin/app.py
from flask import Flask, request
from flask_restful import Resource, Api
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
api = Api(app)
run_with_ngrok(app)

items = []

class Item(Resource):
  def get(self, name):
    for item in items:
      if item['name'] == name:
        return item
    return {"item": None}, 404

  def post(self, name):
    data = request.get_json()
    item = {'name': name, 'price': data['price']}
    items.append(item)
    return item, 201

class ItemList(Resource):
  def get(self):
    return {'items': items}


api.add_resource(Item, '/item/<string:name>')
api.add_resource(ItemList, '/items')

if __name__ == '__main__':
  app.run()

Overwriting bin/app.py


In [ ]:
!python bin/app.py

In [34]:
%%writefile bin/app.py
from flask import Flask, request
from flask_restful import Resource, Api
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
api = Api(app)
run_with_ngrok(app)

items = []

class Item(Resource):
  def get(self, name):
    item = next(filter(lambda x: x['name'] == name, items), None)
    return {"item": item}, 200 if item else 404

  def post(self, name):
    if next(filter(lambda x: x['name'] == name, items), None) is not None:
      return {'message': " an item  '{}' exist".format(name)}, 400

    data = request.get_json()
    item = {'name': name, 'price': data['price']}
    items.append(item)
    return item, 201

class ItemList(Resource):
  def get(self):
    return {'items': items}


api.add_resource(Item, '/item/<string:name>')
api.add_resource(ItemList, '/items')

if __name__ == '__main__':
  app.run()

Overwriting bin/app.py


In [ ]:
!python bin/app.py

In [39]:
%%writefile bin/security.py
from werkzeug.security import safe_str_cmp
from user import User

users = [
    User(1, 'bob', 'asdf'),
    User(2, 'user2', 'abcxyz'),
]

username_table = {u.username: u for u in users}
userid_table = {u.id: u for u in users}

def authenticate(username, password):
    user = username_table.get(username, None)
    if user and safe_str_cmp(user.password, password):
        return user

def identity(payload):
    user_id = payload['identity']
    return userid_table.get(user_id, None)



Overwriting bin/security.py


In [40]:
%%writefile bin/user.py
class User(object):
    def __init__(self, id, username, password):
        self.id = id
        self.username = username
        self.password = password


Overwriting bin/user.py


In [43]:
%%writefile bin/app.py
from flask import Flask, request
from flask_restful import Resource, Api,reqparse
from flask_ngrok import run_with_ngrok
from flask_jwt import JWT, jwt_required

from security import authenticate, identity


app = Flask(__name__)
app.config['PROPAGATE_EXCEPTIONS'] = True # To allow flask propagating exception even if debug is set to false on app
app.secret_key = 'jose'
api = Api(app)
run_with_ngrok(app)

jwt = JWT(app, authenticate, identity)

items = []

class Item(Resource):
    parser = reqparse.RequestParser()
    parser.add_argument('price',
        type=float,
        required=True,
        help="This field cannot be left blank!"
    )

    @jwt_required()
    def get(self, name):
        return {'item': next(filter(lambda x: x['name'] == name, items), None)}

    def post(self, name):
        if next(filter(lambda x: x['name'] == name, items), None) is not None:
            return {'message': "An item with name '{}' already exists.".format(name)}

        data = Item.parser.parse_args()

        item = {'name': name, 'price': data['price']}
        items.append(item)
        return item

    @jwt_required()
    def delete(self, name):
        global items
        items = list(filter(lambda x: x['name'] != name, items))
        return {'message': 'Item deleted'}

    @jwt_required()
    def put(self, name):
        data = Item.parser.parse_args()
        # Once again, print something not in the args to verify everything works
        item = next(filter(lambda x: x['name'] == name, items), None)
        if item is None:
            item = {'name': name, 'price': data['price']}
            items.append(item)
        else:
            item.update(data)
        return item

class ItemList(Resource):
    def get(self):
        return {'items': items}

api.add_resource(Item, '/item/<string:name>')
api.add_resource(ItemList, '/items')

if __name__ == '__main__':
  app.run()

Overwriting bin/app.py


In [ ]:
!python bin/app.py

In [11]:
rm data.db

In [12]:
%%writefile bin/test.py
import sqlite3

connection = sqlite3.connect('data.db')

cursor = connection.cursor()

create_table = "CREATE TABLE users(id int, username text, password text)"

cursor.execute(create_table)

user = (1, 'sumith', 'asdf')
insert_query = "INSERT INTO users VALUES(?,?,?)"
cursor.execute(insert_query, user)

user = [
        (2, 'sumith2', 'asdf'),
        (3, 'amith', 'asdf')
]

cursor.executemany(insert_query, user)

select_query = "Select id, username, password FROM users"
for row in cursor.execute(select_query):
  print(row)

connection.commit()
connection.close()

Writing bin/test.py


In [13]:
!python bin/test.py

(1, 'sumith', 'asdf')
(2, 'sumith2', 'asdf')
(3, 'amith', 'asdf')


In [30]:
%%writefile bin/user.py
import sqlite3


class User(object):
    def __init__(self, id, username, password):
        self.id = id
        self.username = username
        self.password = password

    @classmethod  
    def find_by_username(cls, username):
      connection = sqlite3.connect('data.db')
      cursor = connection.cursor()

      query = "select id, username, password FROM users WHERE username=?"

      print (username)
      result = cursor.execute(query, (username,))
      row = result.fetchone()
      if row:
        user = cls(*row)
      else:
        user = None
      connection.close()
      return user

    @classmethod  
    def find_by_id(cls, username):
      connection = sqlite3.connect('data.db')
      cursor = connection.cursor()

      query = "select id, username, password FROM users WHERE id=?"
      result = cursor.execute(query, (username,))
      row = result.fetchone()
      if row:
        user = cls(*row)
      else:
        user = None
      connection.close()
      return user      

Overwriting bin/user.py


In [31]:
%%writefile bin/security.py
from werkzeug.security import safe_str_cmp
from user import User

def authenticate(username, password):
    user = User.find_by_username(username)
    if user and safe_str_cmp(user.password, password):
        return user

def identity(payload):
    user_id = payload['identity']
    return User.find_by_id(user_id)


Overwriting bin/security.py


In [32]:
%%writefile bin/app.py
from flask import Flask, request
from flask_restful import Resource, Api,reqparse
from flask_ngrok import run_with_ngrok
from flask_jwt import JWT, jwt_required

from security import authenticate, identity


app = Flask(__name__)
app.config['PROPAGATE_EXCEPTIONS'] = True # To allow flask propagating exception even if debug is set to false on app
app.secret_key = 'jose'
api = Api(app)
run_with_ngrok(app)

jwt = JWT(app, authenticate, identity)

items = []

class Item(Resource):
    parser = reqparse.RequestParser()
    parser.add_argument('price',
        type=float,
        required=True,
        help="This field cannot be left blank!"
    )

    @jwt_required()
    def get(self, name):
        return {'item': next(filter(lambda x: x['name'] == name, items), None)}

    def post(self, name):
        if next(filter(lambda x: x['name'] == name, items), None) is not None:
            return {'message': "An item with name '{}' already exists.".format(name)}

        data = Item.parser.parse_args()

        item = {'name': name, 'price': data['price']}
        items.append(item)
        return item

    @jwt_required()
    def delete(self, name):
        global items
        items = list(filter(lambda x: x['name'] != name, items))
        return {'message': 'Item deleted'}

    @jwt_required()
    def put(self, name):
        data = Item.parser.parse_args()
        # Once again, print something not in the args to verify everything works
        item = next(filter(lambda x: x['name'] == name, items), None)
        if item is None:
            item = {'name': name, 'price': data['price']}
            items.append(item)
        else:
            item.update(data)
        return item

class ItemList(Resource):
    def get(self):
        return {'items': items}

api.add_resource(Item, '/item/<string:name>')
api.add_resource(ItemList, '/items')

if __name__ == '__main__':
  app.run()

Overwriting bin/app.py


In [33]:
!python bin/app.py

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Running on http://5777c2383a23.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
sumith
127.0.0.1 - - [18/May/2021 07:27:06] "POST /auth HTTP/1.1" 200 -
^C
